In [1]:
#Instalar el paquete que controla el navegador
!pip install selenium

#Descargar el Driver en :https://sites.google.com/a/chromium.org/chromedriver/downloads

#importar y chequer si selenium funciona

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait     
from selenium.webdriver.common.by import By     
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.select import Select
import openpyxl
from openpyxl import load_workbook
import re
import itertools
import time 
import pandas as pd

# selecionando nuestro webbrowser (selecionar la ubicación donde este el driver)
try:
    driver = webdriver.Chrome("C:/Users/Ivan/Documents/CCCE/WebDriver/chromedriver80")
except:
    driver = webdriver.Chrome("C:/Users/Ivan/Documents/CCCE/WebDriver/chromedriver85")
    print("El driver fue instalado correctamente")

El driver fue instalado correctamente


In [2]:
#pagina donde va a buscar
#Mercadolibre
driver.maximize_window()

docs = openpyxl.load_workbook ("Categorias.xlsx")
hoja = docs.get_sheet_by_name("Mercadolibre")

itemsM = []
for columna in hoja.iter_rows():
    bien = columna[0].value
    itemsM.append(bien)

<ipython-input-2-a347be60feeb>:6: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  hoja = docs.get_sheet_by_name("Mercadolibre")


In [3]:
driver.get("https://www.mercadolibre.com.co/")

In [4]:
linksM = []
priceM = []
descM = []
codesM = []
timeM = []
obj = []
codesMp = []
timeMF = []

from selenium.webdriver.common.keys import Keys

mercadolibre = "https://listado.mercadolibre.com.co/"
dis = "_DisplayType_LF"
for item in itemsM:
    driver.get(mercadolibre + item + dis)
    SCROLL_PAUSE_TIME = 0.5
    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)
        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    bienes = WebDriverWait(driver, 200).until(lambda x: x.find_elements_by_css_selector(".ui-search-layout__item"))
    for elem in bienes:
        pric = elem.find_elements_by_css_selector(".price-tag-fraction")
        for p in pric[0:1]:
            ele = p.text #obtiene el precio del webelement
            value = re.findall(r"[0-9.]*[0-9]+", ele) #quita el signo "$" del precio
            #print(value[0])
            priceM.append(value[0]) #guarda el precio en una lista
    for elem in bienes:
        linksM.append(elem.find_element_by_css_selector("div.ui-search-item__group--title > a").get_attribute('href'))
        timeM.append(time.strftime("%d/%m/%y %H:%M:%S"))
        timeMF.append(time.strftime("%d/%m/%y"))
        obj.append(item)
    #for link in linksM:
        #print(link)
    for elem in bienes:
        descM.append(elem.find_element_by_css_selector("h2.ui-search-item__title").text)
    error = "data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7"
    for elem in bienes:
        if elem.find_element_by_css_selector("div.slick-slide.slick-active > img").get_attribute("src") == error:
            codesMp.append(elem.find_element_by_css_selector("div.slick-slide.slick-active > img").get_attribute("data-src"))
        elif elem.find_element_by_css_selector("div.slick-slide.slick-active > img").get_attribute("src") != error:
            codesMp.append(elem.find_element_by_css_selector("div.slick-slide.slick-active > img").get_attribute("src"))
for link in codesMp: #Obtener Codigo unico de los links
    code = re.findall(r"-MC.(.*?).-V", link)
    #code = des[0]
    codesM.append(code)

In [5]:
len(linksM)

2000

In [6]:
len(priceM)

2000

In [7]:
len(descM)

2000

In [8]:
len(codesM)

2000

In [9]:
len(timeMF)

2000

In [10]:
nowa = time.strftime("%d-%m-%y")
dataf = pd.DataFrame({"Codigo":codesM, "Descripción":descM,
                      "Subcategoria":obj, "Precio":priceM,
                      "Link":linksM, "Fecha & Hora":timeM, "Fecha":timeMF})
book = load_workbook('MercadoLibre.xlsx')
writer = pd.ExcelWriter('MercadoLibre.xlsx', engine = 'openpyxl')
writer.book = book
dataf.to_excel(writer, sheet_name = ("MercadoLibre " + str(nowa)), index = False)
writer.save()

In [2]:
import pandas as pd
xls = pd.ExcelFile(r"C:\Users\Ivan\Documents\CCCE\Mercadolibre\MercadoLibre.xlsx")

In [3]:
names1 = xls.sheet_names

In [7]:
import pandas as pd
import openpyxl
from openpyxl import load_workbook

my_value = pd.DataFrame()
merge_value = pd.DataFrame()
for i in names1:
    my_value = pd.read_excel(r"C:\Users\Ivan\Documents\CCCE\Mercadolibre\MercadoLibre.xlsx", sheet_name = i)
    merge_value = pd.concat([merge_value,my_value], ignore_index = False)
#print(merge_value.head)
merge_value.to_csv("Mercadolibre Consolidado.csv", index = False)

In [10]:
import dropbox
from dropbox.files import WriteMode

cliente = dropbox.Dropbox("sl.Ak-rLRS9DMuGf862Zp0zyMLejTVAeDo110RnEJGmx3Het6osaChMGekVNdPQuAtLC4ugjzelLNXd35c_ROHYIIxPGjjJl7hbM2MaJDIdtQmxaCmmo9VFvBpJREp2DfkAiT4S3MzNkDc")
path = "/Dropbox/Aplicaciones/save_python/Mercadolibre Consolidado.csv"
#print(cliente.users_get_current_account()) 
f= open("C:/Users/Ivan/Documents/CCCE/Mercadolibre/Mercadolibre Consolidado.csv", "rb")
respuesta = cliente.files_upload(f.read(),path, mode = dropbox.files.WriteMode.overwrite)